In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-08 03:02:13--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.99MB/s    in 0.2s    

2022-03-08 03:02:13 (4.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("az_cameras").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://emily-data-bootcamp.s3.amazonaws.com/amazon_reviews_us_Gift_Card_v1_00.tsv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv"), inferSchema=True, sep = '\t')
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [5]:
# Count the number of records (rows) in the dataset.

df.count()

149086

In [6]:
# check the data types 
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
#Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.

# drop incomplete or duplicate rows if there are any

df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())


149080
149080


In [8]:
# select columns to create the dfs that match those in our sql schemas 

# for the review_id_table table
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RR8P8H4144RUD|   28726648|B007V6EVY2|     910961751| 2015-08-31|
|R1QGEWKB99QW3G|   16943853|B00EPLSMGY|     725850175| 2015-08-31|
|R1E6DFR0YQVPCB|   13097243|B00BWDH3VS|     473048287| 2015-08-28|
| RFDF7ZIKH9PVL|   46281218|B00BWDH368|     473048287| 2015-08-25|
| RWGW40MZS35WQ|   29640138|B0066AZGJI|     480731329| 2015-08-22|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [9]:
# change data type of the review_date column (other columns are of the proper data type)
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType

review_id_table = review_id_table.withColumn("review_date",col("review_date").cast(DateType()))
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
# for the products table
products = df.select(["product_id", "product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B007V6EVY2|Amazon Gift Card ...|
|B00EPLSMGY|Amazon eGift Card...|
|B00BWDH3VS|Amazon.com eGift ...|
|B00BWDH368|Amazon.com eGift ...|
|B0066AZGJI|Amazon eGift Card...|
+----------+--------------------+
only showing top 5 rows



In [11]:
# dropping dupes for products table specifically
products = products.dropDuplicates(['product_id', 'product_title'])

In [12]:
# double checking that there are indeed no dupes 
products \
    .groupby(['product_id']) \
    .count() \
    .where('count > 1') \
    .show()

+----------+-----+
|product_id|count|
+----------+-----+
+----------+-----+



In [13]:
# and for the customers table 

customers = df.select(["customer_id"])

customers = customers.groupby("customer_id").agg({"customer_id": "count"})
customers.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   35594973|                 1|
|   18961565|                 1|
|   45120109|                 1|
|   22720037|                 1|
|   45528330|                 1|
+-----------+------------------+
only showing top 5 rows



In [14]:
# rename the count customer id field to customer_count to match the sql schema

customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show(5)


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35594973|             1|
|   18961565|             1|
|   45120109|             1|
|   22720037|             1|
|   45528330|             1|
+-----------+--------------+
only showing top 5 rows



In [15]:
# double checking to make sure that customer_count is of the proper data type
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [16]:
# dropping dupes for customers table specifically
customers = customers.dropDuplicates(['customer_id'])

# double checking that there are indeed no dupes 
customers \
    .groupby(['customer_id']) \
    .count() \
    .where('count > 1') \
    .show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [21]:
# Load the DataFrames that correspond to tables into an RDS instance.

mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}


In [22]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [23]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [24]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)